# Filtering and Restoring of Mixed and Noise Polluted Audio Samples Using Different Approaches

# 1. Edited Audio Samples and Generate Mixed Signals

In [ ]:
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt

inp_folder = 'unedited_samples'
outp_folder = 'audio_samples'

In [ ]:
file_names = ['speech.wav', 'street-noise.wav', 'music.wav', 'white-noise.wav', 'tone_10kHz.wav']

audio_speech, sr_speech = sf.read(os.path.join(inp_folder, file_names[0]))
audio_street, sr_street = sf.read(os.path.join(inp_folder, file_names[1]))
audio_music, sr_music = sf.read(os.path.join(inp_folder, file_names[2]))
audio_wnoise, sr_wnoise = sf.read(os.path.join(inp_folder, file_names[3]))
audio_test_tone, sr_test_tone = sf.read(os.path.join(inp_folder, file_names[4]))
print("Array Type is: {}".format(audio_music.dtype))
# Attenuate white noise 
attenuation_ratio = 1/4
audio_files = [audio_speech, audio_street, audio_music, audio_wnoise * attenuation_ratio, audio_test_tone]
sample_rates = [sr_speech, sr_street, sr_music, sr_wnoise, sr_test_tone]

for name, sr, audio in zip(file_names, sample_rates, audio_files):
    print("File: {}, Sample Rate: {}, Samples: {}, Time: {}sec".format(name, sr, len(audio), len(audio)/sr))

In [ ]:
make_mono = True
# Reduce from stereo to mono
if make_mono:
    for ii, audio in enumerate(audio_files):
        if len(audio.shape) == 2:
            peak_l = max(audio[:, 0])
            peak_r = max(audio[:, 1])
            peak_pre = max(peak_l, peak_r)
            audio_files[ii] = np.sum(audio, axis=1)
            audio_files[ii] /= max(audio_files[ii])
            audio_files[ii] *= peak_pre
            print("Shape File {}: {}".format(ii+1, audio_files[ii].shape))       

In [ ]:
import samplerate

max_sr = max(sample_rates)
converter = 'sinc_best'

for ii, (audio, sr) in enumerate(zip(audio_files, sample_rates)):
    if sr != max_sr:
        ratio = max_sr / sr
        audio_files[ii] = samplerate.resample(audio, ratio, converter)
        print("New Shape: {}".format(audio_files[ii].shape))
        sample_rates[ii] = max_sr
        
for name, audio in zip(file_names, audio_files):
    print("File: {}, Sample Rate: {}, Samples: {}, Time: {}sec".format(name, max_sr, len(audio), len(audio)/sr))

In [ ]:
# Find minimal audio length
audio_length = []
for ii, audio in enumerate(audio_files):
    audio_length.append(len(audio))
max_length = min(audio_length)
print("Minimal number of audio samples: {}".format(max_length))

In [ ]:
# Determine Audio length
# Neede samples are samples = time * sample_rate
time = max_length / sample_rates[audio_length.index(max_length)]

adj_audio_length = []
for sr in sample_rates:
    adj_audio_length.append(int(time * sr))

print("Max audio length is {} s".format(time))
for name, smp_has, smp_should in zip(file_names, audio_length, adj_audio_length):
    print("File {} has {} Samples and must have {}".format(name, smp_has, smp_should))

In [ ]:
# Unify audio file length
for ii, smp in enumerate(adj_audio_length):
    audio_files[ii] = audio_files[ii][:smp]
    print("Shape file {}: {}".format(ii+1, audio_files[ii].shape))

# Time-Domain

Time plot of original audio files $u$ and noise sources $v$.

| Audio | Symbol |
| --- | --- |
| Speech | $u_s$ | 
| Street Noise | $v_{sn}$ |
| Music | $u_m$ |
| White Noise | $v_{wn}$ |

In [ ]:
fig, axs = plt.subplots(len(audio_files), 1, figsize=[18, 10], sharex=False, constrained_layout = True)
audio_names = ['Speech $u_s$', 'Street Noise $v_{sn}$', 'Music $u_m$', 'White Noise $v_{wn}$', 'Tone 10kHz']
colors = ['blue', 'red', 'green', 'black', 'orange']
for ii, (audio, name, sr, color) in enumerate(zip(audio_files, audio_names, sample_rates, colors)):
    x_axis = np.arange(0, len(audio)) / sr
    axs[ii].plot(x_axis, audio, lw=5, color=color)
    axs[ii].set_title(name, fontsize=25)

fig.supylabel('Amplitude', fontsize=20)
fig.supxlabel('Time in Sec', fontsize=20)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

# Spectrum

Frequencies in original audio files.

In [ ]:
# Fast Fourier Transform
from scipy.fftpack import fft, fftfreq

spectrums = []
fft_x = []
for audio, sample_rate, samples in zip(audio_files, sample_rates, adj_audio_length):
    frequencies = fft(audio)
    T = 1/sample_rate
    x_axis = fftfreq(samples, T)[:samples//2]
    spectrums.append(2.0/samples * np.abs(frequencies[0:samples//2]))
    fft_x.append(x_axis)


In [ ]:
fig, axs = plt.subplots(len(audio_files), 1, figsize=[18, 10], sharex=False, constrained_layout = True)

for ii, (spectrum, x_axis, name, color) in enumerate(zip(spectrums, fft_x, audio_names, colors)):
    axs[ii].plot(x_axis / 1e3, spectrum, lw=5, color=color)
    axs[ii].set_title(name, fontsize=25)

fig.supylabel('Magnitude', fontsize=20)
fig.supxlabel('Frequency in kHz', fontsize=20)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

In [ ]:
# Save Files
names_source_out = ['speech.wav', 'street.wav', 'music.wav', 'white_noise.wav', 'tone_10kHz.wav']
for name, audio, sr in zip(names_source_out, audio_files, sample_rates):
    sf.write(os.path.join(outp_folder, name), audio, sr)

# Mix Signals

Generate four observations $d$. 

1. Music + White Noise $d_{m+wn}$
1. Street Noise + Music $d_{sn+m}$
1. Street Noise + Music + Speech $d_{sn+m+s}$
1. Speech + White Noise $d_{s+wn}$

In [ ]:
# 1           Music +       White Noise
d_m_wn = audio_files[2] + audio_files[3]
# 2           Street +      Music
d_sn_m = audio_files[1] + audio_files[2]
# 3 :         Street +      Music +             Speech
d_sn_m_s = audio_files[1] + audio_files[2] + audio_files[0]
# 4 :         Speech +      White Noise
d_s_wn = audio_files[0] + audio_files[3]

observations = [d_m_wn, d_sn_m, d_sn_m_s, d_s_wn]


In [ ]:
# Save Files
outp_folder = 'mixed_signals'
names_out = ['music_wn.wav', 'street_music.wav', 'street_music_speech.wav', 'speech_wn.wav']
for name, audio, sr in zip(names_out, observations, sample_rates):
    sf.write(os.path.join(outp_folder, name), audio, sr)